<a href="https://colab.research.google.com/github/linnea-backgard/DataScienceProjectSpring23/blob/main/projekt_diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
# Importera bibliotek
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.impute import SimpleImputer


from sklearn.pipeline import Pipeline

# Modeller
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.svm import SVC
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV

# Metrics
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import make_scorer

from sklearn.metrics import ConfusionMatrixDisplay

# SMOTE för oversampling
#from imblearn.over_sampling import SMOTE



In [84]:

from google.colab import files
 
 
uploaded = files.upload()

Saving diabetes2.csv to diabetes2 (4).csv


In [85]:

import pandas as pd
import io
 
df = pd.read_csv(io.BytesIO(uploaded['diabetes2.csv']))
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [29]:
# Visar 50 översta rader för att upptäcka nollvärden också
df.head(50)
# Visar information om datasetet
df.info()
# Visar hur många rader och kolumner
df.shape
# Kollar hur många nollvärden det finns i varje kolumn.
(df == 0).sum()
# Kollar hur många nollvärden det finns i hela dataframen.
(df == 0).sum().sum()
# Är detta så många värden som finns i hela dataframen????
df.sum().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


276660.701

(768, 9)

Kollar referensvärden - insulin: https://emedicine.medscape.com/article/2089224-overview
Referensvärden glukos
Adult - 74-106 mg/dL or 4.1-5.9 mmol/L, For elderly patients, the normal reference ranges are as follows [1] : 60-90 years - 82-115 mg/dL or 4.6-6.4 mmol/L >90 years - 75-121 mg/dL or 4.2-6.7 mmol/L

In [86]:
# Dela upp datan träning och test
X = df.drop('Outcome', axis=1)
y = df['Outcome']

In [87]:
# Delar i tränings och testset med stratify för att behålla proportionerna mellan klasserna
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, stratify=y, random_state=42)

In [88]:
X_train.head(50)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
353,1,90,62,12,43,27.2,0.580,24
711,5,126,78,27,22,29.6,0.439,40
373,2,105,58,40,94,34.9,0.225,25
46,1,146,56,0,0,29.7,0.564,29
682,0,95,64,39,105,44.6,0.366,22
360,5,189,64,33,325,31.2,0.583,29
414,0,138,60,35,167,34.6,0.534,21
339,7,178,84,0,0,39.9,0.331,41
694,2,90,60,0,0,23.5,0.191,25
33,6,92,92,0,0,19.9,0.188,28


In [57]:
y_train

353    0
711    0
373    0
46     0
682    0
      ..
451    1
113    0
556    0
667    1
107    0
Name: Outcome, Length: 614, dtype: int64

In [89]:
# Gör en ny variabel som innehåller både X_train och y_train
train_df = pd.concat([X_train, y_train], axis=1)

train_df.head(50)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
353,1,90,62,12,43,27.2,0.580,24,0
711,5,126,78,27,22,29.6,0.439,40,0
373,2,105,58,40,94,34.9,0.225,25,0
46,1,146,56,0,0,29.7,0.564,29,0
682,0,95,64,39,105,44.6,0.366,22,0
360,5,189,64,33,325,31.2,0.583,29,1
414,0,138,60,35,167,34.6,0.534,21,1
339,7,178,84,0,0,39.9,0.331,41,1
694,2,90,60,0,0,23.5,0.191,25,0
33,6,92,92,0,0,19.9,0.188,28,0


In [90]:

# Gör lista som innehåller kolumnerna som har nollvärden.
columns = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
df = train_df

# Funktion för att rökna ut olika kolumners medianer
def filter_no_zero_values_median(df, columns):
    
    medians_class_0 = []
    medians_class_1 = []
    
    for col in columns:
        not_zero_outcome_0 = df[(df[col] != 0) & (df['Outcome'] == 0)][col]
        not_zero_outcome_1 = df[(df[col] != 0) & (df['Outcome'] == 1)][col]
        median_outcome_0 = np.median(not_zero_outcome_0)
        median_outcome_1 = np.median(not_zero_outcome_1)
        medians_class_0.append(median_outcome_0)
        medians_class_1.append(median_outcome_1)
   
    return (medians_class_0, medians_class_1)

train_df_medians = filter_no_zero_values_median(df, columns)

print(train_df_medians)

# Utveckla funktionen till kanske dictionairy eller så det blir tydligae när man skriver ut

([108.0, 70.0, 27.0, 100.0, 30.1, 0.3415, 27.0], [141.5, 76.0, 33.0, 168.0, 34.349999999999994, 0.461, 36.0])


In [91]:
df = train_df
medians_class_0, medians_class_1 = filter_no_zero_values_median(df, columns)

def impute_with_median(df, columns, medians_class_0, medians_class_1):
    
    # Imputera medianvärdet där det finns nollvärden i df
    for col in columns:
        df.loc[(df[col] == 0) & (df['Outcome'] == 0), col] = medians_class_0[columns.index(col)]
        df.loc[(df[col] == 0) & (df['Outcome'] == 1), col] = medians_class_1[columns.index(col)]
    
    return (df)

df_imputed = impute_with_median(df, columns, train_df_medians[0], train_df_medians[1])

print(df_imputed.head(50))

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin    BMI  \
353            1     90.0             62             12       43  27.20   
711            5    126.0             78             27       22  29.60   
373            2    105.0             58             40       94  34.90   
46             1    146.0             56             27      100  29.70   
682            0     95.0             64             39      105  44.60   
360            5    189.0             64             33      325  31.20   
414            0    138.0             60             35      167  34.60   
339            7    178.0             84             33      168  39.90   
694            2     90.0             60             27      100  23.50   
33             6     92.0             92             27      100  19.90   
218            5     85.0             74             22      168  29.00   
705            6     80.0             80             36      100  39.80   
76             7     62.0

In [92]:
columns = ['Glucose', 'BloodPressure', 'SkinThickness','Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
df = df_imputed

medians_df_imputed = filter_no_zero_values_median(df, columns)

print(medians_df_imputed)

([108.0, 70.0, 27.0, 100.0, 30.1, 0.3415, 27.0], [141.5, 76.0, 33.0, 168.0, 34.349999999999994, 0.461, 36.0])


In [93]:
# Kollar hur många nollvärden det finns i varje kolumn.
(train_df == 0).sum()

Pregnancies                  85
Glucose                       0
BloodPressure                 0
SkinThickness                 0
Insulin                       0
BMI                           0
DiabetesPedigreeFunction      0
Age                           0
Outcome                     400
dtype: int64

In [94]:
X_train = df_imputed.drop('Outcome', axis=1)

X_train_imputed = X_train

In [95]:
# Create a SimpleImputer object with strategy='median'

imp_median = SimpleImputer(strategy='median')

print(imp_median)

# Fit the imputer on the training set
imp_median.fit(X_train)

# Transform the test set
X_test_imputed = pd.DataFrame(imp_median.transform(X_test), columns=X_train.columns)

# Print the median of all columns in the training set

for col in X_train_imputed.columns:
  print('Median {} value in the training set: {}'.format(col, X_train_imputed[col].median()))

SimpleImputer(strategy='median')
Median Pregnancies value in the training set: 3.0
Median Glucose value in the training set: 117.0
Median BloodPressure value in the training set: 72.0
Median SkinThickness value in the training set: 28.0
Median Insulin value in the training set: 100.0
Median BMI value in the training set: 32.4
Median DiabetesPedigreeFunction value in the training set: 0.3825
Median Age value in the training set: 29.0


In [96]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_imputed)

In [97]:
X_test_scaled = scaler.transform(X_test)

In [98]:
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
353,1,90.0,62,12,43,27.2,0.580,24
711,5,126.0,78,27,22,29.6,0.439,40
373,2,105.0,58,40,94,34.9,0.225,25
46,1,146.0,56,27,100,29.7,0.564,29
682,0,95.0,64,39,105,44.6,0.366,22


In [99]:

scaled_X_train

array([[-0.85135507, -1.05708598, -0.82841991, ..., -0.76616897,
         0.31079384, -0.79216928],
       [ 0.35657564,  0.14257662,  0.47455755, ..., -0.4144692 ,
        -0.11643851,  0.56103382],
       [-0.5493724 , -0.55722656, -1.15416427, ...,  0.36220112,
        -0.76486207, -0.70759409],
       ...,
       [-0.85135507, -0.82381825, -0.17693118, ...,  0.83113414,
        -0.78607218, -0.28471812],
       [ 1.86648903, -0.35728279, -0.17693118, ..., -0.7222065 ,
        -1.01938346,  0.56103382],
       [ 0.05459296,  0.74240792, -1.15416427, ..., -0.42912336,
        -0.57700104,  0.30730824]])

In [125]:

# Definierar modellen med ökad max_iter
log_reg = LogisticRegression(solver="lbfgs", max_iter=1000, random_state=42)

# Sätter hyperparametrarna
params = {"C": [0.1, 1, 10], "penalty": ["l2"]}

# Skapa en instans av GridSearchCV
log_reg_cv = GridSearchCV(log_reg, params, cv=5, scoring=recall_scorer)


# Träna modellen med cross-validation
log_scores = cross_val_score(log_reg_cv, X_train_scaled, y_train, cv=5)
print("Cross-validation scores: {}".format(log_scores))
print("Average cross-validation score: {:.2f}".format(log_scores.mean()))

# Fit modellen på hela datamängden med bästa uppsättningen av hyperparametrar
log_reg_cv.fit(X_train_scaled, y_train)

print("Best parameters: {}".format(log_reg_cv.best_params_))
print("Best cross-validation score: {:.2f}".format(log_reg_cv.best_score_))

# Prediktera testdata och utvärdera modellen
y_pred_log_reg = log_reg_cv.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred_log_reg)
precision = precision_score(y_test, y_pred_log_reg)
recall = recall_score(y_test, y_pred_log_reg)

print(f'Accuracy score log_reg: {accuracy}')
print(f'Precision score log_reg: {precision}')
print(f'Recall score log_reg: {recall}')

Cross-validation scores: [0.65116279 0.62790698 0.60465116 0.55813953 0.5952381 ]
Average cross-validation score: 0.61
Best parameters: {'C': 1, 'penalty': 'l2'}
Best cross-validation score: 0.61
Accuracy score log_reg: 0.6558441558441559
Precision score log_reg: 0.5161290322580645
Recall score log_reg: 0.2962962962962963


### LogisticRegression: Bästa crossvalidation score 0.61. Hamnade då på 0.65 i testset och 0.29 i recall. 

In [112]:
dt_clf = DecisionTreeClassifier(random_state=42)
dt_params = {"max_depth": [1, 3, 5]}
dt_cv = GridSearchCV(dt_clf, dt_params, cv=5, scoring=recall_scorer)
dt_scores = cross_val_score(dt_cv, X_train_scaled, y_train, cv=5)
print("Decision Tree Cross-validation scores: {}".format(dt_scores))
print("Average cross-validation score: {:.2f}".format(dt_scores.mean()))
dt_cv.fit(X_train_scaled, y_train)

print("Best parameters: {}".format(dt_cv.best_params_))
print("Best cross-validation score: {:.2f}".format(dt_cv.best_score_))

# Prediktera testdata och utvärdera modellen
y_pred_dt = dt_cv.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred_dt)
precision = precision_score(y_test, y_pred_dt)
recall = recall_score(y_test, y_pred_dt)

print(f'Accuracy score dt: {accuracy}')
print(f'Precision score dt: {precision}')
print(f'Recall score dt: {recall}')

Decision Tree Cross-validation scores: [0.79069767 0.93023256 0.88372093 0.8372093  0.88095238]
Average cross-validation score: 0.86
Best parameters: {'max_depth': 1}
Best cross-validation score: 0.86
Accuracy score dt: 0.6038961038961039
Precision score dt: 0.4
Recall score dt: 0.25925925925925924


### DecisionTree: Bästa crossvalidation score 0.86. Hamnade då på 0.60 på testset och bara 0.25 i recall. 

In [119]:
# Definierar modellen
rf_clf = RandomForestClassifier(random_state=42)

# Sätter parametrar
param_grid = {'n_estimators': [60, 70, 80],
              'max_depth': [4, 5, 7]}

recall_scorer = make_scorer(recall_score)

rf_grid_search = GridSearchCV(rf_clf, param_grid, cv=5, scoring=recall_scorer)
rf_grid_search.fit(X_train_scaled, y_train)

print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(rf_grid_search.best_score_))


Best parameters: {'max_depth': 7, 'n_estimators': 80}
Best cross-validation score: 0.83


In [120]:
# Prediktera och utvärdera modellen på träningssetet
y_pred_rf_train = rf_grid_search.predict(X_train_scaled)
accuracy = accuracy_score(y_train, y_pred_rf_train)
precision = precision_score(y_train, y_pred_rf_train)
recall = recall_score(y_train, y_pred_rf_train)

print(f'Accuracy score rf: {accuracy}')
print(f'Precision score rf: {precision}')
print(f'Recall score rf: {recall}')

Accuracy score rf: 0.9788273615635179
Precision score rf: 0.9808612440191388
Recall score rf: 0.9579439252336449


In [123]:
# Prediktera testdata och utvärdera modellen
y_pred_rf_test = rf_grid_search.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred_rf_test)
precision = precision_score(y_test, y_pred_rf_test)
recall = recall_score(y_test, y_pred_rf_test)

print(f'Accuracy score rf: {accuracy}')
print(f'Precision score rf: {precision}')
print(f'Recall score rf: {recall}')

Accuracy score rf: 0.6558441558441559
Precision score rf: 0.5217391304347826
Recall score rf: 0.2222222222222222


### RandomForestClassifier : Bästa crossvalidation score 0.83. Hamnade då på 0.65 på testset och bara 0.22 i recall.

In [114]:
# Definiera modellen
svm_clf = SVC()

# Sätt parametrar
param_grid = {'C': [0.1, 1, 10],
              'gamma': [0.001, 0.01, 0.1 ]}

recall_scorer = make_scorer(recall_score)

svm_grid_search = GridSearchCV(svm_clf, param_grid, cv=5, scoring=recall_scorer)
svm_grid_search.fit(X_train_scaled, y_train)

print("Best parameters: {}".format(svm_grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(svm_grid_search.best_score_))


Best parameters: {'C': 1, 'gamma': 0.1}
Best cross-validation score: 0.71


In [117]:
# Prediktera testdata och utvärdera modellen på träningssetet
y_pred_svc_train = svm_grid_search.predict(X_train_scaled)
accuracy = accuracy_score(y_train, y_pred_svc_train)
precision = precision_score(y_train, y_pred_svc_train)
recall = recall_score(y_train, y_pred_svc_train)

print(f'Accuracy score svc: {accuracy}')
print(f'Precision score svc: {precision}')
print(f'Recall score svc: {recall}')

Accuracy score svc: 0.8941368078175895
Precision score svc: 0.8669950738916257
Recall score svc: 0.822429906542056


In [118]:
# Prediktera testdata och utvärdera modellen på testset
y_pred_svc_test = svm_grid_search.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred_svc_test)
precision = precision_score(y_test, y_pred_svc_test)
recall = recall_score(y_test, y_pred_svc_test)

print(f'Accuracy score svc: {accuracy}')
print(f'Precision score svc: {precision}')
print(f'Recall score svc: {recall}')

Accuracy score svc: 0.6688311688311688
Precision score svc: 0.5517241379310345
Recall score svc: 0.2962962962962963


### SVC: Bästa crossvalidation score 0.71. Hamnade då på 0.66 i testset och 0.29 i recall.

In [129]:
from sklearn.linear_model import SGDClassifier


# Definiera modellen
sgd_clf = SGDClassifier(random_state=42)

# Sätter parametrar
param_grid = {'alpha': [0.00001, 0.0001, 0.01],
              'penalty': ['l1', 'l2', 'elasticnet']}

recall_scorer = make_scorer(recall_score)

sgd_grid_search = GridSearchCV(sgd_clf, param_grid, cv=5, scoring=recall_scorer)
sgd_grid_search.fit(X_train_scaled, y_train)

print("Best parameters: {}".format(sgd_grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(sgd_grid_search.best_score_))


Best parameters: {'alpha': 1e-05, 'penalty': 'l2'}
Best cross-validation score: 0.73


In [130]:
# Prediktera testdata och utvärdera modellen på träningssetet
y_pred_sgd_train = sgd_grid_search.predict(X_train_scaled)
accuracy = accuracy_score(y_train, y_pred_sgd_train)
precision = precision_score(y_train, y_pred_sgd_train)
recall = recall_score(y_train, y_pred_sgd_train)

print(f'Accuracy score svc: {accuracy}')
print(f'Precision score svc: {precision}')
print(f'Recall score svc: {recall}')

Accuracy score svc: 0.7752442996742671
Precision score svc: 0.6938775510204082
Recall score svc: 0.6355140186915887


In [131]:
# Prediktera testdata och utvärdera modellen på testset
y_pred_sgd_test = sgd_grid_search.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred_sgd_test)
precision = precision_score(y_test, y_pred_sgd_test)
recall = recall_score(y_test, y_pred_sgd_test)

print(f'Accuracy score svc: {accuracy}')
print(f'Precision score svc: {precision}')
print(f'Recall score svc: {recall}')

Accuracy score svc: 0.6623376623376623
Precision score svc: 0.5294117647058824
Recall score svc: 0.3333333333333333


### SGD: Bästa crossvalidation score 0.73. Hamnade sen på 0.66 med 0.33 i recall. Bäst hittills

In [25]:
models = {}
# Support Vector Machines
from sklearn.svm import LinearSVC
models['Support Vector Machines'] = LinearSVC()
# Logistic Regression
from sklearn.linear_model import LogisticRegression
models['Logistic Regression'] = LogisticRegression()
# Decision Trees
from sklearn.tree import DecisionTreeClassifier
models['Decision Trees'] = DecisionTreeClassifier()
# Random Forest
from sklearn.ensemble import RandomForestClassifier
models['Random Forest'] = RandomForestClassifier()
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
models['Naive Bayes'] = GaussianNB()
# K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
models['K-Nearest Neighbor'] = KNeighborsClassifier()

In [ ]:
# Initierar DecisionTreeClassifier
d_tree_clf = DecisionTreeClassifier(random_state=42)

# Setting hyperparameters
params = {"max_leaf_nodes": list(range(40, 70, 10)), "min_samples_leaf": [3, 4, 5]}

# GridSearch
d_tree_clf = GridSearchCV(d_tree_clf, params, n_jobs=-1, cv=3, verbose=1)

# Tränar modellen på datan 
d_tree_clf.fit(X_train, y_train)

print(d_tree_clf.best_estimator_)

In [ ]:
# Predikterar och mäter hur bra modellen blivit
y_pred_dt = d_tree_clf.predict(X_test)

print(f'Accuracy score dt: {accuracy_score(y_test, y_pred_dt)}')

In [ ]:
metrics.ConfusionMatrixDisplay.from_predictions(y_test, y_pred_d_tree)

In [ ]:
# Initierar SGDClassifier
sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)

# Tränar modellen
sgd_clf.fit(X_train, y_train)

In [ ]:
# Predikterar och mäter hur bra modellen blivit
y_pred_sgd = sgd_clf.predict(X_test)

print(f'Accuracy score SGD: {accuracy_score(y_test, y_pred_sgd)}')
print(f'RMSE: {np.sqrt(metrics.mean_squared_error(y_test, y_pred_sgd))}')

In [ ]:
# Initierar LogisticRegression
log_reg = LogisticRegression(solver="lbfgs", random_state=42)

log_reg.fit(X_train, y_train)

In [ ]:
# Predikterar och mäter hur bra modellen blivit
y_pred_log = log_reg.predict(X_test)

print(f'Accuracy score LogisticRegresion: {accuracy_score(y_test, y_pred_log)}')

In [26]:
accuracy, precision, recall, F1, ROC_auc = {}, {}, {}, {}, {}
for key in models.keys():
    
    # Fit the classifier
    models[key].fit(X_train, y_train)
       
    # Make predictions
    predictions = models[key].predict(X_train)
    
    # Calculate metrics
    accuracy[key] = accuracy_score(predictions, y_train)
    precision[key] = precision_score(predictions, y_train)
    recall[key] = recall_score(predictions, y_train)
    F1[key] = f1_score(predictions, y_train)
    ROC_auc[key]= roc_auc_score(predictions, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [27]:
# Build the KNN model
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)

# Train the model on the training set
knn.fit(X_train, y_train)

# Make predictions on the testing set
Y_pred = knn.predict(X_train)

# Calculate evaluation metrics
accuracy = accuracy_score(y_train, y_pred)
precision = precision_score(y_train, y_pred)
recall = recall_score(y_train, y_pred)
f1 = f1_score(y_train, y_pred)
roc_auc = roc_auc_score(y_train, y_pred)
total = accuracy + precision + recall + f1 + roc_auc

# Print the evaluation metrics with a header and the values under it
print('{:<15} {:<15}'.format('Metric', 'Value'))
print('{:<15} {:.2f}'.format('Accuracy', accuracy))
print('{:<15} {:.2f}'.format('Precision', precision))
print('{:<15} {:.2f}'.format('Recall', recall))
print('{:<15} {:.2f}'.format('F1 score', f1))
print('{:<15} {:.2f}'.format('ROC AUC score', roc_auc))
print('{:<15} {}'.format('Total', round(total, 2)))

Metric          Value          
Accuracy        0.48
Precision       0.07
Recall          0.04
F1 score        0.05
ROC AUC score   0.38
Total           1.03
